# A simple Gemini agent in Google Colab

This is a simple demonstration of an AI agent that is using the [gemini-2.0-flash](https://deepmind.google/technologies/gemini/flash/) LLM to answer questions about the latest stats of crypto currencies.

### Install required libraries

The first library we need to install is the Python SDK for the Gemini API,
 contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package.

 Additionally, we will need to install the [`requests`](https://pypi.org/project/requests/) library in order to be able to make HTTP requests to the [CoinMarketCap API](https://pro.coinmarketcap.com/api/v1?source=post_page---------------------------#operation/getV2CryptocurrencyQuotesLatest)

In [ ]:
!pip install -q -U google-generativeai requests

## Load the secret API keys

Now, we need to load the API keys for the Gemini and the CoinMarketCap APIs. You can generate your own keys for Gemini [here](https://aistudio.google.com/app/apikey) and for CoinMarketCap [here](https://pro.coinmarketcap.com/account).

Once you have your keys, add them in your secrets, on the left side panel of your Google Colab project. Make sure the names match the snippet below.

In [ ]:
from google.colab import userdata


GEMINI_API_KEY=userdata.get("GEMINI_API_KEY")
COINMARKETCAP_API_KEY=userdata.get("COINMARKETCAP_API_KEY")

## Create a tool function

Tools are essentially functions that can be used by the agent in order to enhance the LLMs capabilities.

In this case, we can create a function that, given a list of crypto currency names, makes a request to the CoinMarketCap API and returns a response.

In [ ]:
import json
from typing import Dict, List

from pydantic import BaseModel
import requests


COINMARKETCAP_API_URL = "https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest"

class CryptoCurrency(BaseModel):
    name: str
    price: float
    total_supply: float
    market_cap: float


def get_currencies(currency_names: List[str]) -> List[CryptoCurrency]:
    """Given a list of crypto currency names, it makes a request to the
    CoinMarketCap API and returns a list of currency objects with their
    latest details, such as prices.

    Args:
        currency_names: A list of crypto currency names to retrieve.

    Returns:
        A list of CryptoCurrency objects with their latest details.
    """
    params = {"slug": ",".join(currency_names)}
    headers = {
        "Accepts": "application/json",
        "X-CMC_PRO_API_KEY": COINMARKETCAP_API_KEY,
    }

    crypto_currencies = []
    try:
        response = requests.get(COINMARKETCAP_API_URL, params=params, headers=headers)
        data = json.loads(response.text)

        for _, currency_data in data["data"].items():
            crypto_currencies.append(
                CryptoCurrency(
                    name=currency_data["name"],
                    price=currency_data["quote"]["USD"]["price"],
                    total_supply=currency_data["total_supply"],
                    market_cap=currency_data["quote"]["USD"]["market_cap"],
                )
            )

    except Exception as exc:
        print(exc)

    return crypto_currencies

You can read more about function calling [here](https://ai.google.dev/gemini-api/docs/function-calling).

## Instantiate the agent

Now, we can instantiate the agent by choosing the required LLM model and providing the add-on tool that we created.

In [ ]:
from google import genai
from google.genai.types import FunctionDeclaration, GenerateContentConfig, Part, Tool


client = genai.Client(api_key=GEMINI_API_KEY)
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=GenerateContentConfig(
        tools=[
            get_currencies
        ]
    )
)

## Create an output formatter [Optional]

This is an optional step. It's is just a simple function that can be used to convert the output of the LLM to Markdown format so that it is displayed a bit more nicely in the Colab notebook.

In [ ]:
import textwrap

from IPython.display import Markdown

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Interact with the agent

Finally, we can interact with the agent by sending messages.

In [ ]:
text = "Give me a summary of the latest stats for ethereum and bitcoin today"
response = chat.send_message(text)
to_markdown(response.text)

> Here's a summary of the latest stats for Bitcoin and Ethereum:
> 
> **Bitcoin:**
> 
> *   Price: $90877.53
> *   Market Cap: $1,802,366,934,215.69
> *   Total Supply: 19,832,921
> 
> **Ethereum:**
> 
> *   Price: $2283.39
> *   Market Cap: $275,365,843,884.19
> *   Total Supply: 120,594,948.93


In [ ]:
text = "Can you convert the prices in USD?"
response = chat.send_message(text)
to_markdown(response.text)

> The prices I provided for Bitcoin and Ethereum are already in United States Dollars (USD). There is no conversion needed.
